In [11]:
import torch
import torchvision
import numpy as np
import random
import os

# 데이터 준비

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split

label_df = pd.read_csv("./data/train.csv")

train_label_df, valid_label_df = train_test_split(label_df, test_size=0.1, stratify=label_df["has_cactus"], random_state=42)

len(train_label_df), len(valid_label_df)

(15750, 1750)

In [13]:
import cv2
import torch.utils

class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, label_df, image_dir, transform):
        super(ImageDataset, self).__init__()

        self.label_df = label_df
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.label_df)
    
    def __getitem__(self, idx):
        image_file_name = self.label_df.iloc[idx, 0] # id: 파일명
        image_file_path = f"{self.image_dir}/{image_file_name}"
        image = cv2.imread(image_file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image = self.transform(image)
        
        label = self.label_df.iloc[idx, 1] # has_cactus

        return image, label
    
transform = torchvision.transforms.ToTensor()

train_dataset = ImageDataset(label_df=train_label_df, image_dir="./data/train", transform=transform)
valid_dataset = ImageDataset(label_df=valid_label_df, image_dir="./data/train", transform=transform)

train_data_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
valid_data_loader = torch.utils.data.DataLoader(dataset=valid_dataset, batch_size=32, shuffle=True)

## Baseline Model

In [14]:
import torch.nn as nn

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()

        self.layer_1 = nn.Sequential(nn.Conv2d(in_channels=3, out_channels=32, 
                                               kernel_size=3, padding=2),
                                     nn.ReLU(),
                                     nn.MaxPool2d(kernel_size=2))

        self.layer_2 = nn.Sequential(nn.Conv2d(in_channels=32, out_channels=64,
                                               kernel_size=3, padding=2),
                                     nn.ReLU(),
                                     nn.MaxPool2d(kernel_size=2))
        
        self.avg_pool = nn.AvgPool2d(kernel_size=2)

        self.fc = nn.Linear(in_features=64*4*4, out_features=2)

    def forward(self, x):
        # x: 32x32 3
        x = self.layer_1(x) # 17x17 32
        x = self.layer_2(x) # 9x9 64
        x = self.avg_pool(x) # 4x4 64
        x = x.view(-1, 64*4*4) # flatten
        x = self.fc(x)
        return x

model = Model()
model

Model(
  (layer_1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer_2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avg_pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc): Linear(in_features=1024, out_features=2, bias=True)
)

# Train

In [23]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

epochs = 10

for epoch in range(epochs):
    epoch_loss = 0

    for images, labels in train_data_loader:
        optimizer.zero_grad()
        
        # forward
        outputs = model(images)
        loss = criterion(outputs, labels)

        # backward
        epoch_loss += loss.item()
        loss.backward()

        optimizer.step()
    
    print(f"Epoch [{epoch+1}/{epochs}] - loss: {epoch_loss/len(train_data_loader):.4f}")

Epoch [1/10] - loss: 0.1455
Epoch [2/10] - loss: 0.1329
Epoch [3/10] - loss: 0.1096
Epoch [4/10] - loss: 0.0992
Epoch [5/10] - loss: 0.0902
Epoch [6/10] - loss: 0.0816
Epoch [7/10] - loss: 0.0824
Epoch [8/10] - loss: 0.0774
Epoch [9/10] - loss: 0.0689
Epoch [10/10] - loss: 0.0724


# Evaluate

In [24]:
from sklearn.metrics import roc_auc_score

true_list = [] # 실제값
preds_list = [] # 예측값

model.eval() # 평가 상태로 변경

with torch.no_grad(): # 기울기 계산 비활성화
    for images, labels in valid_data_loader:
        outputs = model(images)
        preds = torch.softmax(outputs, dim=1)[:, 1] # 선인장이 포함될 확률을 가져온다
        true = labels

        preds_list.extend(preds)
        true_list.extend(true)

print(f"ROC AUC: {roc_auc_score(true_list, preds_list):.4f}")

ROC AUC: 0.9921


# Submission

In [39]:
submission_df = pd.read_csv("./data/sample_submission.csv")
test_dataset = ImageDataset(label_df=submission_df, image_dir="./data/test", transform=transform)
# shuffle=False 주의!
test_data_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

In [40]:
preds_list = []

model.eval() # 평가 상태로 변경

with torch.no_grad(): # 기울기 계산 비활성화
    for images, _labels in test_data_loader:
        outputs = model(images)
        preds = torch.softmax(outputs, dim=1)[:, 1].tolist() # 선인장이 포함될 확률을 가져온다
        preds_list.extend(preds)

preds_list[:5]

[0.9892098307609558,
 0.9999980926513672,
 0.023401642218232155,
 0.026309529319405556,
 0.8063020706176758]

In [42]:
submission_df["has_cactus"] = preds_list
submission_df.to_csv("./data/baseline_submission.csv", index=False)